In [2]:
# Import numpy 
import numpy as np

In [4]:
def RK4_integration(x1, step_size, interp_phi_prime):
    
    '''
    Defines RK4-integration scheme for computing null-geodesics.
    
    Parameters:
        x1:               array(3, Npoints), position and angles (#Npoints = Number of initial conditions).
                          x1[0,:] contains x-positions
                          x1[1,:] contains y-positions
                          x1[2,:] contains angle phi
        step_size:        float, step size used for integration. This value is kept constant.
        interp_phi_prime: Interpolant-object for phi_prime
        
    Returns:
        x_update:         array(3, Npoints), updated position (#Npoints = Number of initial conditions)
    '''
    
    # Compute x_prime at the beginning of the time-step by re-orienting and rescaling the vector field
    x_prime = _diff_equation(x1, interp_phi_prime)
    
    # compute derivative
    k1 = step_size * x_prime

    #  position and time at the first midpoint.
    x2 = x1 + .5 * k1
        
    # Compute x_prime at the first midpoint.
    x_prime = _diff_equation(x2, interp_phi_prime)
    
    # compute derivative
    k2 = step_size * x_prime

    # Update position at the second midpoint.
    x3 = x1 + .5 * k2
    
    # Compute x_prime at the second midpoint.
    x_prime = _diff_equation(x3, interp_phi_prime)
    
    # compute derivative
    k3 = step_size * x_prime
    
    # Update position at the endpoint.
    x4 = x1 + k3
    
    # Compute derivative at the end of the time-step.
    x_prime = _diff_equation(x4, interp_phi_prime) 
    
    # compute derivative
    k4 = step_size * x_prime
    
    # Compute RK4 derivative
    x_prime_update = 1.0 / 6.0*(k1 + 2 * k2 + 2 * k3 + k4)
    
    # Integration x <-- x + x_prime*ds
    x_update = x1 + x_prime_update

    return x_update

In [5]:
def _diff_equation(x_phi, interp_phi_prime):
    '''
    Defines differential equation for computing null-geodesics.
    
    Parameters:
        x_phi:            array(3, Npoints), position and angles (#Npoints = Number of initial conditions).
                          x_phi[0,:] x-positions
                          x_phi[1,:] y-positions
                          x_phi[2,:] angle phi
        interp_phi_prime: Interpolant-object for phi_prime 
        
    Returns:
        RHS:              array(3, Npoints), x_phi_prime (#Npoints = Number of initial conditions). 
                          This is the (normalized) right-hand-side of the differential equation.
    '''
    
    # Position of particle in the extended phase space of (x, y, phi)
    x, y, phi = x_phi[0,:].ravel(), x_phi[1,:].ravel(), x_phi[2,:].ravel()%(2*np.pi)   
    
    # Compute phi_prime @ (x, y, phi)
    phi_dot = interp_phi_prime(np.array([y, x, phi]).transpose())
    
    # x' = (cos(phi), sin(phi))
    x_dot = np.cos(phi)
    y_dot = np.sin(phi)
    
    # normalize the RHS of the differential equation for numerical stability of RK45
    norm = np.sqrt(1+phi_dot**2)
    
    RHS = np.array([x_dot/norm, y_dot/norm, phi_dot/norm])
    
    return RHS